<a href="https://colab.research.google.com/github/mmsamiei/world/blob/main/world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
!pip install Faker

In [131]:
from faker import Faker
import random
import datetime
from itertools import filterfalse

In [132]:
fake = Faker()

In [133]:
class Location:

  def __init__(self, map_size=50):
    self.x =  random.uniform(-map_size, map_size)
    self.y = random.uniform(-map_size, map_size)

  def distance(self, other_location):
    dx = other_location.x - self.x
    dy = other_location.y - self.y
    return (dx**2+dy**2)**0.5

In [134]:
class Person:
  
  def __init__(self, world):

    self.world = world
    self.gender = random.choices(['M', 'F'])
    self.first_name = fake.first_name_female() if self.gender == 'F' else fake.first_name_male()
    self.last_name = fake.last_name()
    self.birthday = self.world.date
    self.deathday = None
    self.location = Location()
    self.velocity = 10
    self.energy = 100

  def __str__(self):
    x = round(self.location.x,2)
    y = round(self.location.y,2)
    age = self.world.date - self.birthday

    return f"{self.first_name} {self.last_name}  at {x}, {y} energy: {self.energy}  age: {age}"
  
  def choose_action(self):
    
    for food in self.world.foods:
      if self.location.distance(food.location) < self.world.distance_can_eat:
        return 'eat', (self, food)

    day_to_death = self.energy / self.world.hunger_penalty
    dx = 0
    dy = 0
    for food in self.world.foods:
      if self.location.distance(food.location)/self.velocity < day_to_death:
        dx = food.location.x - self.location.x
        dy = food.location.y - self.location.y
        length = (dx**2+dy**2)**0.5
        dx_norm = (dx / length) * (self.velocity ** 0.5)
        dy_norm = (dy / length) * (self.velocity ** 0.5)
        if dx**2 + dy**2 < dx_norm**2 + dy_norm**2:
          dir_x, dir_y = dx, dy
        else:
          dir_x, dir_y= dx_norm, dy_norm
        break
    return 'move', (self, dx, dy)

In [135]:
class Food:

  def __init__(self, world):
    self.world = world
    self.location = Location()
    self.produce_date = self.world.date

In [136]:
class World:

  def __init__(self, name):
    self.name = name
    self.persons = []
    self.dead_persons = []
    self.date = datetime.date(2018, 6, 1)
    self.foods = []
    self.hunger_penalty = 20
    self.distance_can_eat = 5
    self.every_day_foods = 100

  def simulate_day(self):
    self.date += datetime.timedelta(days=1)

    new_person = Person(world=self)
    self.persons.append(new_person)

    for i in range(self.every_day_foods):
      self.foods.append(Food(self))
    self.foods[:] = filterfalse(lambda f: (world.date - f.produce_date).days > 5 , self.foods)

    for person in self.persons[:]:
      self.simulate_person(person)
  

  def simulate_person(self, person):
    person.energy -= self.hunger_penalty
    action, args = person.choose_action()
    
    if action == 'eat':
      food = args[1]
      self.foods.remove(food)
      person.energy = 100
    elif action == 'move':
      dx, dy = args[1], args[2]
      person.location.x += dx
      person.location.y += dy

    if person.energy == 0 :
      self.persons.remove(person)
      self.dead_persons.append(person)
      person.deathday = self.date

  def simulate(self, until=365*10):
    for i in range(until):
      self.simulate_day()
      #print(len(self.foods))
      print("persons ",len(self.persons), len(self.dead_persons))
      


In [137]:
world = World('mahdi_world')
world.simulate()

persons  1 0
persons  2 0
persons  3 0
persons  4 0
persons  5 0
persons  6 0
persons  7 0
persons  8 0
persons  9 0
persons  10 0
persons  11 0
persons  12 0
persons  13 0
persons  14 0
persons  15 0
persons  16 0
persons  17 0
persons  18 0
persons  19 0
persons  20 0
persons  21 0
persons  22 0
persons  23 0
persons  24 0
persons  25 0
persons  26 0
persons  27 0
persons  28 0
persons  29 0
persons  30 0
persons  31 0
persons  32 0
persons  33 0
persons  34 0
persons  35 0
persons  36 0
persons  37 0
persons  38 0
persons  39 0
persons  40 0
persons  41 0
persons  42 0
persons  43 0
persons  44 0
persons  45 0
persons  46 0
persons  47 0
persons  48 0
persons  49 0
persons  50 0
persons  51 0
persons  52 0
persons  53 0
persons  54 0
persons  55 0
persons  56 0
persons  57 0
persons  58 0
persons  59 0
persons  60 0
persons  61 0
persons  62 0
persons  63 0
persons  64 0
persons  65 0
persons  61 5
persons  62 5
persons  63 5
persons  64 5
persons  65 5
persons  66 5
persons  67 5
p

In [138]:
for person in world.persons:
  print(person)

Richard Robertson  at 11.69, 6.32 energy: 80  age: 3649 days, 0:00:00
Michael Steele  at -34.2, -18.49 energy: 80  age: 3648 days, 0:00:00
Brian Hanson  at 26.83, -5.61 energy: 100  age: 3647 days, 0:00:00
Jason Obrien  at 17.45, 2.15 energy: 100  age: 3646 days, 0:00:00
Adam Reilly  at -45.06, -9.86 energy: 100  age: 3645 days, 0:00:00
William Gomez  at 12.9, -40.19 energy: 80  age: 3644 days, 0:00:00
David Mitchell  at -22.73, -3.23 energy: 100  age: 3643 days, 0:00:00
Edward Pham  at 12.9, -40.19 energy: 80  age: 3642 days, 0:00:00
Joseph Lane  at 12.9, -40.19 energy: 80  age: 3641 days, 0:00:00
Stephen Ibarra  at 0.18, 8.64 energy: 100  age: 3640 days, 0:00:00
Nicholas Parker  at -20.94, -12.28 energy: 100  age: 3638 days, 0:00:00
Ralph Mcmillan  at -14.94, -5.73 energy: 100  age: 3635 days, 0:00:00
Jonathan Boyd  at -24.66, -26.56 energy: 100  age: 3342 days, 0:00:00
Stephen Williams  at -11.97, -20.14 energy: 100  age: 1767 days, 0:00:00
Richard Thompson  at 0.18, 8.64 energy: 10

In [143]:
a = datetime.date(2018, 3,3)
b = datetime.date(2013, 6,1)

In [146]:
a-b

datetime.timedelta(days=1736)

In [148]:
a.year - b.year

5